## 获取仓库名

In [1]:
import requests
from bs4 import BeautifulSoup  
import re  
import urllib.request, urllib.error  

# 得到指定一个URL的网页内容
def askURL(url):
    head = {  
        "User-Agent": "Mozilla / 5.0(Windows NT 10.0; Win64; x64) AppleWebKit / 537.36(KHTML, like Gecko) Chrome / 80.0.3987.122  Safari / 537.36"
    }

    request = urllib.request.Request(url, headers=head)
    html = ""
    try:
        response = urllib.request.urlopen(request)
        html = response.read().decode("utf-8", errors='ignore')
    except urllib.error.URLError as e:
        if hasattr(e, "code"):
            print(e.code)
        if hasattr(e, "reason"):
            print(e.reason)
    return html

In [ ]:
# 获取第2页及之后仓库名
baseurl = 'https://notabug.org/explore/repos?page=' # 截至2023年10月24日，notabug有897页仓库数据，在第2页及以后所有的url链接为此格式
repo_names = []
for i in range(2, 900):
    url = baseurl + str(i) + '&q='
    #print(url)
    html = askURL(url)  # 保存获取到的网页源码
    soup = BeautifulSoup(html, "html.parser")
    # 使用 find_all 方法找到所有带有 class='name' 的 a 标签
    links = soup.find_all('a', class_='name')
    # 提取 href 属性值
    href_values = [link.get('href') for link in links]
    repo_names.extend(href_values)

In [5]:
# 获取第1页仓库名
url = 'https://notabug.org/explore/repos' # 第1页仓库url链接
html = askURL(url)  
soup = BeautifulSoup(html, "html.parser")
links = soup.find_all('a', class_='name')
href_values = [link.get('href') for link in links]
repo_names.extend(href_values)

In [6]:
# 去重
repo_names = list(set(repo_names))

In [8]:
with open("repo_names.txt", "w") as file:
    for item in repo_names:
        file.write(f"{item}\n")

## 获取仓库对应下载链接

In [2]:
# 从文件读取列表
with open("repo_names.txt", "r") as file:
    repo_names = [line.strip() for line in file]
base_url = 'https://notabug.org'
download_url = []

In [4]:
print_log = open("printlog.txt",'w')

In [ ]:
for i, repo_name in enumerate(repo_names):
    if i != 17228:
        continue
    url = base_url + repo_name
    html = askURL(url) 
    soup = BeautifulSoup(html, 'lxml')
    # 找到包含链接的div标签
    dropdown_div = soup.find('div', class_='ui basic jump dropdown icon button')
    
    # 提取所有class为"item"的a标签
    try:
        items = dropdown_div.find_all('a', class_='item')
        # 遍历每个a标签，打印链接和文本内容
        for item in items:
            link = item['href']
            link = base_url + link
            print(link, file=print_log)
            download_url.append(link)
    except AttributeError: # 仓库没有对应下载链接
        print('FLAG', file=print_log)
        print(repo_name, file=print_log)

In [ ]:
print_log.close()

In [16]:
file_path = 'printlog.txt'
with open(file_path, "r") as file:
    download_url = [line.strip() for line in file]

In [21]:
zip_urls = []
for zip_url in download_url:
    if len(zip_url.split(sep='/'))==7 and zip_url[-4:]=='.zip':
        zip_urls.append(zip_url)

In [23]:
with open("zip_urls.txt", "w") as file:
    for item in zip_urls:
        file.write(f"{item}\n")